## next-generation matrix for SEIR equations

Some material taken from [here](https://web.stanford.edu/~jhj1/teachingdocs/Jones-Epidemics050308.pdf), but modified/further explained.

Suppose we have the SEIR model, with natural mortality ($\mu$) but without vital dynamics (i.e., no births/entry in to the susceptible class, not that this really matters for the current computation):

$$
\begin{split}
\frac{dS}{dt} & = -\beta S I \\
\frac{dE}{dt} & = \beta S I - (\mu + \sigma) E \\
\frac{dI}{dt} & = \sigma E - (\mu + \gamma) I \\
\frac{dR}{dt} & = \gamma I - \mu R
\end{split}
$$

$$
\newcommand{\F}{\cal F}
\newcommand{\Vm}{\cal V^{-}}
$$
$$
\newcommand{\Vp}{\cal V^{+}}
\newcommand{\V}{\cal V}
$$

For the next-generation matrix, we want to identify the infected compartments ($E$ and $I$) and the transition sets $\F$ (infection), $\Vm$ (movement out of infected compartments), and $\Vp$ ("rate of transfer of individuals into [infected compartments] by all other means"); $\V \equiv \Vm-\Vp$. *This is the tricky part.*

We will define
$$
\begin{split}
\F & = \{ \beta S I, 0 \} \\
\Vm & = \{ -(\sigma+\mu) E, -(\gamma+\mu) I \} \\
\Vp & = \{ 0, \sigma E \}
\end{split}
$$
now we want the Jacobians of $\F$ and $\V$, evaluated at the disease-free equilibrium ($\{S=N,E=I=R=0\}$) denoted $F$ and $V$ respectively.
We get:
$$
\begin{split}
F& = \left(\begin{array}{cc} 0 & \beta N \\ 0 & 0 \end{array} \right)
\\
V & = \left(\begin{array}{cc} \sigma+\mu & 0 \\ -\sigma & \gamma+\mu \end{array} \right)
\end{split}
$$

so
$$
V^{-1} = \left(\begin{array}{cc} 1/(\sigma+\mu) & 0 \\ \sigma/((\gamma+\mu)(\sigma+\mu)) & 1/(\gamma+\mu) \end{array} \right)
$$

and

$$
G = F V^{-1} = \left(\begin{array}{cc} \beta\sigma N/((\sigma+\mu)(\gamma+\mu)) & \beta N/(\gamma+\mu) \\ 0 & 0 \end{array} \right)
$$

... so it's clear that the eigenvalues are $G_{11}=\frac{\beta\sigma N}{(\sigma+\mu)(\gamma+\mu)}$ and 0; since all the parameters are positive, $G_{11}$ is the spectral radius (largest eigenvalue), which corresponds to ${\cal R}_0$.

If we were doing this the old-fashioned/informal way, we would argue as follows: "the rate at which a single infectious person generates new infections in a population of susceptibles of size $N$ (we don't worry about $N$ vs $N-1$ in this context) is $\beta S I = (\beta N) \cdot 1 = \beta N$, but we have to discount this by the fraction $\sigma/(\sigma + \mu)$ that survive the exposed period to become infectious. The expected duration of the infectious period is the inverse of the *per capita* rate at which infectious people leave the $I$ class, or $1/(\gamma+\mu)$, so we have
$$
\newcommand{\fsig}{\frac{\sigma}{\sigma+\mu}}
{\cal R}_0 = \underbrace{\beta N \vphantom{\fsig}}_{\text{new infections}} \cdot \underbrace{\fsig}_{\text{discounting}} \cdot
\underbrace{\frac{1}{\gamma+\mu}}_{\text{infectious duration}} \quad .
$$

## lazy linear algebra

If we're too lazy or careful to get the linear algebra right, we can use `sympy`:

In [1]:
from sympy import *
beta, N, mu, sigma, gamma = symbols("beta N mu sigma gamma")

In [17]:
F = Matrix([[0,beta*N],[0,0]])
V = Matrix([[sigma+mu,0],[-sigma,gamma+mu]])

In [20]:
print(V**(-1))  ## sympy uses M**(-1) for matrix inversion

Matrix([[1/(mu + sigma), 0], [sigma/((gamma + mu)*(mu + sigma)), 1/(gamma + mu)]])


In [18]:
G = F*(V**(-1))
print(G)

Matrix([[N*beta*sigma/((gamma + mu)*(mu + sigma)), N*beta/(gamma + mu)], [0, 0]])


In [19]:
G.eigenvals()

{0: 1, N*beta*sigma/((gamma + mu)*(mu + sigma)): 1}